In [7]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import matplotlib.pyplot as plt
import os
import math

# Create output directories
os.makedirs("forecasts", exist_ok=True)
os.makedirs("plots", exist_ok=True)
os.makedirs("evaluations", exist_ok=True)

# Load and prepare data
df = pd.read_csv("E:\Agriculture project\Data\Processed\Merged_data_features.csv", parse_dates=["Date"])
df.columns = df.columns.str.strip()
df.set_index("Date", inplace=True)
df.index = pd.to_datetime(df.index, errors="coerce")
df = df[df.index.notna()]

# Parameters
forecast_horizon = 60
sequence_length = 24

# Identify unique regions
regions = {col.split('_')[0] for col in df.columns if col.endswith(("_rain", "_soil", "_temp"))}

for region in regions:
    rain_col, soil_col, temp_col = f"{region}_rain", f"{region}_soil", f"{region}_temp"
    if rain_col not in df or soil_col not in df or temp_col not in df:
        print(f"Skipping {region}: Missing data")
        continue

    region_data = df[[rain_col, soil_col, temp_col]].dropna()
    if len(region_data) < sequence_length + forecast_horizon:
        print(f"Skipping {region}: Not enough data")
        continue

    # Scale data
    feature_scaler = MinMaxScaler()
    target_scaler = MinMaxScaler()
    scaled_features = feature_scaler.fit_transform(region_data)
    scaled_targets = target_scaler.fit_transform(region_data)

    # Create sequences
    X, y = [], []
    for i in range(len(scaled_features) - sequence_length):
        X.append(scaled_features[i:i+sequence_length])
        y.append(scaled_targets[i+sequence_length])

    X, y = np.array(X), np.array(y)
    train_size = int(len(X) * 0.8)
    X_train, X_test = X[:train_size], X[train_size:]
    y_train, y_test = y[:train_size], y[train_size:]

    # Build LSTM model
    model = Sequential([
        LSTM(128, return_sequences=True, input_shape=(sequence_length, X.shape[2])),
        Dropout(0.3),
        LSTM(624),
        Dropout(0.3),
        Dense(3)  # Changed from 2 to 3 to include temperature
    ])

    model.compile(optimizer='adam', loss='mse')
    early_stopping = EarlyStopping(monitor='val_loss', patience=15, restore_best_weights=True)

    # Train model
    history = model.fit(
        X_train, y_train, 
        epochs=150, 
        batch_size=32, 
        validation_data=(X_test, y_test), 
        callbacks=[early_stopping],
        verbose=1
    )

    # Evaluate model on test data
    y_pred_scaled = model.predict(X_test)
    y_pred = target_scaler.inverse_transform(y_pred_scaled)
    y_test_actual = target_scaler.inverse_transform(y_test)
    
    # Calculate metrics for each variable
    rain_rmse = math.sqrt(mean_squared_error(y_test_actual[:, 0], y_pred[:, 0]))
    rain_mae = mean_absolute_error(y_test_actual[:, 0], y_pred[:, 0])
    rain_r2 = r2_score(y_test_actual[:, 0], y_pred[:, 0])
    
    soil_rmse = math.sqrt(mean_squared_error(y_test_actual[:, 1], y_pred[:, 1]))
    soil_mae = mean_absolute_error(y_test_actual[:, 1], y_pred[:, 1])
    soil_r2 = r2_score(y_test_actual[:, 1], y_pred[:, 1])
    
    temp_rmse = math.sqrt(mean_squared_error(y_test_actual[:, 2], y_pred[:, 2]))
    temp_mae = mean_absolute_error(y_test_actual[:, 2], y_pred[:, 2])
    temp_r2 = r2_score(y_test_actual[:, 2], y_pred[:, 2])
    
    # Store evaluation metrics in a DataFrame and save to CSV
    eval_metrics = {
        'Metric': ['RMSE', 'MAE', 'R²'],
        'Rainfall': [rain_rmse, rain_mae, rain_r2],
        'Soil_Moisture': [soil_rmse, soil_mae, soil_r2],
        'Temperature': [temp_rmse, temp_mae, temp_r2]
    }
    
    eval_df = pd.DataFrame(eval_metrics)
    eval_df.to_csv(f"evaluations/{region}_evaluation.csv", index=False)
    
    # Generate forecasts
    last_sequence = X[-1].reshape(1, sequence_length, X.shape[2])
    future_predictions = []
    for _ in range(forecast_horizon):
        pred = model.predict(last_sequence)[0]
        future_predictions.append(pred)
        last_sequence = np.vstack([last_sequence[0][1:], pred]).reshape(1, sequence_length, X.shape[2])

    # Convert predictions back to original scale
    future_predictions = target_scaler.inverse_transform(future_predictions)
    future_dates = pd.date_range(region_data.index[-1] + pd.DateOffset(months=1), periods=forecast_horizon, freq="M")
    
    # Create forecast DataFrame that includes evaluation metrics
    forecast_df = pd.DataFrame({
        "Date": future_dates, 
        f"{rain_col}_forecast": future_predictions[:, 0], 
        f"{soil_col}_forecast": future_predictions[:, 1],
        f"{temp_col}_forecast": future_predictions[:, 2]  # Add temperature forecast
    })
    
    # Add evaluation metrics as metadata at the top of the file
    with open(f"forecasts/{region}_forecast.csv", "w") as f:
        f.write(f"# Evaluation Metrics for {region}\n")
        f.write(f"# Rainfall RMSE: {rain_rmse:.4f}, MAE: {rain_mae:.4f}, R²: {rain_r2:.4f}\n")
        f.write(f"# Soil Moisture RMSE: {soil_rmse:.4f}, MAE: {soil_mae:.4f}, R²: {soil_r2:.4f}\n")
        f.write(f"# Temperature RMSE: {temp_rmse:.4f}, MAE: {temp_mae:.4f}, R²: {temp_r2:.4f}\n")
    
    # Append the forecast data
    forecast_df.to_csv(f"forecasts/{region}_forecast.csv", index=False, mode='a')

    # Plot results with evaluation metrics in the title
    # Rainfall plot
    plt.figure(figsize=(14, 6))
    plt.plot(region_data.index[-100:], region_data[rain_col].values[-100:], label='Historical Rainfall', color='blue')
    plt.plot(future_dates, future_predictions[:, 0], label='Forecast Rainfall', color='red', linestyle='--')
    plt.legend()
    plt.title(f'Rainfall Forecast for {region}\nRMSE: {rain_rmse:.4f}, R²: {rain_r2:.4f}')
    plt.savefig(f"plots/{region}_rainfall_forecast.png")
    plt.close()

    # Soil moisture plot
    plt.figure(figsize=(14, 6))
    plt.plot(region_data.index[-100:], region_data[soil_col].values[-100:], label='Historical Soil Moisture', color='green')
    plt.plot(future_dates, future_predictions[:, 1], label='Forecast Soil Moisture', color='orange', linestyle='--')
    plt.legend()
    plt.title(f'Soil Moisture Forecast for {region}\nRMSE: {soil_rmse:.4f}, R²: {soil_r2:.4f}')
    plt.savefig(f"plots/{region}_soil_forecast.png")
    plt.close()
    
    # Temperature plot - Added for temperature
    plt.figure(figsize=(14, 6))
    plt.plot(region_data.index[-100:], region_data[temp_col].values[-100:], label='Historical Temperature', color='purple')
    plt.plot(future_dates, future_predictions[:, 2], label='Forecast Temperature', color='brown', linestyle='--')
    plt.legend()
    plt.title(f'Temperature Forecast for {region}\nRMSE: {temp_rmse:.4f}, R²: {temp_r2:.4f}')
    plt.savefig(f"plots/{region}_temperature_forecast.png")
    plt.close()

    # Create and save actual vs. predicted plots for the test set
    test_dates = region_data.index[train_size+sequence_length:train_size+sequence_length+len(y_test)]
    
    # Rainfall test validation
    plt.figure(figsize=(14, 6))
    plt.plot(test_dates, y_test_actual[:, 0], label='Actual Rainfall', color='blue')
    plt.plot(test_dates, y_pred[:, 0], label='Predicted Rainfall', color='red', linestyle='--')
    plt.legend()
    plt.title(f'Rainfall Test Set Predictions for {region}\nRMSE: {rain_rmse:.4f}, R²: {rain_r2:.4f}')
    plt.savefig(f"plots/{region}_rainfall_test_validation.png")
    plt.close()
    
    # Soil moisture test validation
    plt.figure(figsize=(14, 6))
    plt.plot(test_dates, y_test_actual[:, 1], label='Actual Soil Moisture', color='green')
    plt.plot(test_dates, y_pred[:, 1], label='Predicted Soil Moisture', color='orange', linestyle='--')
    plt.legend()
    plt.title(f'Soil Moisture Test Set Predictions for {region}\nRMSE: {soil_rmse:.4f}, R²: {soil_r2:.4f}')
    plt.savefig(f"plots/{region}_soil_test_validation.png")
    plt.close()
    
    # Temperature test validation - Added for temperature
    plt.figure(figsize=(14, 6))
    plt.plot(test_dates, y_test_actual[:, 2], label='Actual Temperature', color='purple')
    plt.plot(test_dates, y_pred[:, 2], label='Predicted Temperature', color='brown', linestyle='--')
    plt.legend()
    plt.title(f'Temperature Test Set Predictions for {region}\nRMSE: {temp_rmse:.4f}, R²: {temp_r2:.4f}')
    plt.savefig(f"plots/{region}_temperature_test_validation.png")
    plt.close()

print("Forecasting and evaluation completed.")

Epoch 1/150
12/12 ━━━━━━━━━━━━━━━━━━━━ 21s 552ms/step - loss: 0.0788 - val_loss: 0.0514
Epoch 2/150
12/12 ━━━━━━━━━━━━━━━━━━━━ 5s 385ms/step - loss: 0.0440 - val_loss: 0.0491
Epoch 3/150
12/12 ━━━━━━━━━━━━━━━━━━━━ 5s 417ms/step - loss: 0.0419 - val_loss: 0.0528
Epoch 4/150
12/12 ━━━━━━━━━━━━━━━━━━━━ 6s 538ms/step - loss: 0.0450 - val_loss: 0.0458
Epoch 5/150
12/12 ━━━━━━━━━━━━━━━━━━━━ 10s 483ms/step - loss: 0.0371 - val_loss: 0.0444
Epoch 6/150
12/12 ━━━━━━━━━━━━━━━━━━━━ 5s 451ms/step - loss: 0.0316 - val_loss: 0.0306
Epoch 7/150
12/12 ━━━━━━━━━━━━━━━━━━━━ 5s 420ms/step - loss: 0.0221 - val_loss: 0.0317
Epoch 8/150
12/12 ━━━━━━━━━━━━━━━━━━━━ 5s 419ms/step - loss: 0.0226 - val_loss: 0.0273
Epoch 9/150
12/12 ━━━━━━━━━━━━━━━━━━━━ 5s 450ms/step - loss: 0.0228 - val_loss: 0.0271
Epoch 10/150
12/12 ━━━━━━━━━━━━━━━━━━━━ 5s 413ms/step - loss: 0.0214 - val_loss: 0.0266
Epoch 11/150
12/12 ━━━━━━━━━━━━━━━━━━━━ 6s 457ms/step - loss: 0.0200 - val_loss: 0.0242
Epoch 12/150
12/12 ━━━━━━━━━━━━━━━━━━━━